In [ ]:
#Standard inmorts 
import numpy as np
import pandas as pd
import os
import feather
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import pysubgroup as ps

from scipy import io
from scipy import stats



#the rest should be ML 

from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# K-Means 
from sklearn.cluster import KMeans


# Pipeline stuff
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

## settings

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 1. Read Data

In [ ]:
df1 = pd.read_feather('../Data3/D3_EBAN_CL.ftr', columns=None, use_threads=True)

### Data inspection

In [ ]:
df1.head(5)

In [ ]:
print(df1.shape)

In [ ]:
df1['kproto_labels'] = df1['kproto_labels'].astype('category')
df1['dbscan_gower'] = df1['dbscan_gower'].astype('category')
df1['AgloSngl_gower'] = df1['AgloSngl_gower'].astype('category')
df1['AgloAvg_gower'] = df1['AgloAvg_gower'].astype('category')
df1['AgloComp_gower'] = df1['AgloComp_gower'].astype('category')

In [ ]:
df1.dtypes

# Filtering

In [ ]:
df2 = df1.drop(['EBELN','EBELP','BANFN'], axis=1).copy()

In [ ]:
df2.head(20)

# precompute Dataframe

In [ ]:
df_kproto = pd.concat([df2.iloc[: , :-5], df2.loc[:, ['kproto_labels']]],axis=1)

In [ ]:
# label festlegen
df_kproto['target'] = df_kproto['kproto_labels'].copy()
df_kproto = df_kproto.drop('kproto_labels', axis= 1).copy()

In [ ]:
df_kproto.head(2)

In [ ]:
df_dbscan = pd.concat([df2.iloc[: , :-5], df2.loc[:, ['dbscan_gower']]],axis=1)
# label festlegen
df_dbscan['target'] = df_dbscan['dbscan_gower'].copy()
df_dbscan = df_dbscan.drop('dbscan_gower', axis= 1).copy()

In [ ]:
df_dbscan.head(2)

In [ ]:
df_aglo = pd.concat([df2.iloc[: , :-5], df2.loc[:, ['AgloAvg_gower']]],axis=1)
# label festlegen
df_aglo['target'] = df_aglo['AgloAvg_gower'].copy()
df_aglo = df_aglo.drop('AgloAvg_gower', axis= 1).copy()

In [ ]:
df_aglo.head(2)

In [ ]:
df_aglo.dtypes

# Analysis

In [ ]:
# festlegen was wir analysieren
df = df_aglo.copy()

In [ ]:
l_catCols = list(df.select_dtypes(include=['category']).columns)
l_numCols = list(df.select_dtypes(include=['float64', 'int64']).columns)

#l_catCols = df.select_dtypes('category').columns.to_list()
#l_numCols = ['MENGE', 'PREIS']
#l_allCols = df.columns.to_list()

In [ ]:
# possible colors
mypal2 =['#FEAEFE', '#FC05FB', '#B4FFE4', '#3FFEBA', '#1F363D', '#2B4570', '#3A2D32', '#706F6F','#FC05FB']
mypal= ['#FC05FB', '#FEAEFE', '#FCD2FC','#F3FEFA', '#B4FFE4','#3FFEBA']

#set color based on subgroups
print(df['target'].nunique())
if df['target'].nunique() == 2:
    #asdf
    colorss = [mypal2[i] for i in [0,3]]
elif df['target'].nunique() == 3:
    #asdf
    colorss = [mypal2[i] for i in [0,3,5]]
elif df['target'].nunique() == 4:
    colorss = [mypal2[i] for i in [0,1,3,5]]
elif df['target'].nunique() == 5:
    colorss = [mypal2[i] for i in [0,1,2,3,5]]
elif df['target'].nunique() == 6:
    colorss = [mypal2[i] for i in [0,1,2,3,5,6]]
elif df['target'].nunique() == 7:
    colorss = [mypal2[i] for i in [0,1,2,3,4,5,6]]
elif df['target'].nunique() == 8:
    colorss = [mypal2[i] for i in [0,1,2,3,4,5,6,7]]
elif df['target'].nunique() == 9:
    colorss = [mypal2[i] for i in [0,1,2,3,4,5,6,7,8]]

In [ ]:
nf = l_numCols + ['target']

data_ = df[nf].copy()

g = sns.pairplot(data_, hue="target", corner=True, diag_kind='hist', palette=colorss) # palette="pastel"
sns.set_context("paper")
#sns.set_style("darkgrid")
#plt.suptitle('Numerical features pairplot' ,fontsize = 16)
plt.subplots_adjust(top=1)

In [ ]:
#export
#g.savefig('exports/EBAN_EDA_num.pdf')  

In [ ]:
#sns.reset_defaults() 
sns.reset_orig()

In [ ]:
cat_feats = l_catCols.copy()
data = df.copy()
cat_feats.remove('TXZ01')
#cat_feats.remove('LMEIN')

def hist_plot(data, cat_feats):    
    L = len(cat_feats)
    ncol= 2
    nrow= int(np.ceil(L/ncol))
    remove_last= (nrow * ncol) - L

    fig, ax = plt.subplots(nrow, ncol, figsize=(8, 12))    #, facecolor='#F6F5F4'
    fig.subplots_adjust(top=0.92)
    #fig.legend('test', loc='lower center')
    #ax.flat[-remove_last].set_visible(False)
    #sns.set_style("darkgrid")
    sns.set_context("paper")

    i = 1
    for col in cat_feats:
        plt.subplot(nrow, ncol, i) #facecolor='#F6F5F4'
        h = sns.countplot(data=data, x=col, hue="target", palette=colorss) #, palette="pastel" oder eigene farben, palette=colorss 
        #sns.set_style("darkgrid")
        #sns.set_context("paper")
        
        h.set_xlabel(col)
        h.set_ylabel("count") #fontsize=18
        sns.despine(right=True)
        sns.despine(offset=0, trim=False)
        sns.set_context("paper") # paper
        #plt.legend(facecolor='#F6F5F4')
        #legende bei allen außer erstem subplot hiden
        
        if i == 2:
            plt.legend(loc = 2, bbox_to_anchor = (1,1))
            
        else:
            plt.legend('',frameon=False)
        i = i +1
        
    

    #plt.suptitle('cluster distribution in Categorical features count-plot' ,fontsize = 16)
    #export
    
    #fig.savefig('exports/EBAN_EDA_cat.pdf')  
    return 0
    
        
    
hist_plot(data, cat_feats[0:-1]);

# reduce coliniarity

In [ ]:
df_corc = df.copy()

cats_cat = df_corc[l_catCols]
        
for (columnName, columnData) in cats_cat.iteritems():
    df_corc[columnName] = df_corc[columnName].astype(str)
    if isinstance(df_corc[columnName][0], str):
        s = pd.Series(df_corc[columnName], dtype="category")
        df_corc[columnName] = s.cat.rename_categories(np.arange(len(np.unique(df_corc[columnName]))))

In [ ]:
# Compute the corr matrix and 
corr = df_corc.astype(int).corr()

# Set up the triangular mask / matplotlib figure / Colors
mask = np.triu(np.ones_like(corr, dtype=bool)) # tril for upper
f, ax = plt.subplots(figsize=(15, 15))
cmap1 = sns.cubehelix_palette(dark=0, light=1, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap1, vmax=1,vmin=-1, center=0, square=True, annot=True, linewidths=.5, cbar_kws={"shrink": .5})


In [ ]:
# delete highly correlating features
import fg_cleansing_helper

#get cols to drop through correlation analysis 
to_drop = fg_cleansing_helper.getCorrelatedCols(df_corc.astype(int), cut = 0.95)
to_drop.remove('target')
# drop cols in both DFs
df_corc = df_corc.drop(to_drop, axis=1).copy()
df = df.drop(to_drop, axis=1).copy()

In [ ]:
# Compute the corr matrix and 
corr1 = df_corc.astype(int).corr()

# Set up the triangular mask / matplotlib figure / Colors
mask = np.triu(np.ones_like(corr1, dtype=bool)) # tril for upper
f, ax = plt.subplots(figsize=(15, 15))
cmap1 = sns.cubehelix_palette(dark=0, light=1, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr1, mask=mask, cmap=cmap1, vmax=1, vmin=-1, center=0, square=True, annot=True, linewidths=.5, cbar_kws={"shrink": .5})

# Visualisation

In [ ]:
import pacmap
from sklearn.preprocessing import StandardScaler

In [ ]:
df_corc.head()

In [ ]:
df_fa = df_corc.copy() 

# df_fa = df_corc.drop(['STATU', 'BEDAT', 'FLIEF', 'target', 'MEINS'],axis=1).copy() 
#.drop(['MEINS', 'LMEIN', 'EKGRP'],axis=1).copy()

## Calc Eig. Values to determine Dimensions

In [ ]:
from factor_analyzer import FactorAnalyzer
# Create factor analysis object and perform factor analysis
fa = FactorAnalyzer()
fa.set_params(n_factors=28, rotation=None)
fa.fit(df_fa)
#fa.analyze(df, 25, rotation=None)
# Check Eigenvalues
#loadings = fa.loadings_
ev, v = fa.get_eigenvalues()
ev

In [ ]:
df_pca = df_corc.copy()

## Tests

In [ ]:
df_tests = StandardScaler().fit_transform(df_corc.copy())

In [ ]:
# Import library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

X = df_fa.astype(float)
calc_vif(X)

### Bartlett’s sphericity test

The Bartlett’s sphericity test measures if there are
significant difference between the correlation matrix and the
identify matrix (perfect correlation). In this case, value p <
0.05 therefore, PCA is applicable

In [ ]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
chi_square_value,p_value=calculate_bartlett_sphericity(df_fa.astype(float))
chi_square_value, p_value

### Kaiser-Meyer-Olkin (KMO) 

Another measure, Kaiser-Meyer-Olkin (KMO), indicate
how well suited the dataset with the PCA. In this case, the
value of the Measure of Sampling Adequacy (MSA) is
just ~0.6, means that PCA could be useful.

- 0.00 to 0.49 unacceptable.
- 0.50 to 0.59 miserable.
- 0.60 to 0.69 mediocre.
- 070 to 0.79 middling.
- 080 to 0.89 meritorious.
- 0.90 to 1.00 marvelous.


In [ ]:
from factor_analyzer.factor_analyzer import calculate_kmo
kmo_all,kmo_model=calculate_kmo(df_fa.astype(float))

In [ ]:
kmo_model

## Dim Red

In [ ]:
df_pca.head()

In [ ]:
df_pca2 = df_pca.drop(['target'], axis=1)
scaled_df = StandardScaler().fit_transform(df_pca2)

In [ ]:
#'PaCMAP': pacmap.PaCMAP()
reducer2D =pacmap.PaCMAP(n_components=2)
embedding2D = reducer2D.fit_transform(scaled_df)


In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter(
    embedding2D,
    width=400, 
    height=400,
    x=0, y=1,
    color=df_pca['target'].astype(str), # alternativ .astype(int)
    labels={'0':' ', '1':' ', 'color': 'cluster'},
    #color_continuous_scale='Viridis',
    #color_continuous_scale=px.colors.sequential.Viridis oinly if not astype(str
    color_discrete_map={'0': '#FEAEFE', 
                        '1': '#3FFEBA',
                        '2': '#2B4570', 
                        '3': '#FC05FB'},
    template="simple_white"
)
fig.update_traces(marker_size=7)
fig.show()
fig.write_image('exports/EBAN_clust_agglo.pdf')  

In [ ]:
#'PaCMAP': pacmap.PaCMAP()
reducer3D =pacmap.PaCMAP(n_components=3)
embedding3D = reducer3D.fit_transform(scaled_df)

In [ ]:
#show results in 3D 
fig = px.scatter_3d(
    embedding3D,
    width=800, 
    height=800,
    x=0, y=1, z=2,
    color=df_pca['target'].astype(str), # alternativ .astype(int)
    labels={'color': 'cluster'},
    #color_continuous_scale='Viridis',
    #color_continuous_scale=px.colors.sequential.Viridis oinly if not astype(str
    color_discrete_map={'0': '#FEAEFE', 
                        '1': '#3FFEBA',
                        '2': '#2B4570', 
                        '3': '#FC05FB'},
)
fig.update_traces(marker_size=5)
fig.show()
fig.write_image('exports/EBAN_clust_agglo_3D.pdf') 

In [ ]:
import plotly.express as px
dfpc = df_pca.drop(['BEDAT'], axis=1).astype('float64').copy()
fig = px.parallel_coordinates(dfpc, 
                              color="target",
                              color_continuous_scale=px.colors.diverging.Tealrose, 
                              color_continuous_midpoint=2)
fig.show()